# XR2Text: Model Training with HAQT-ARR

This notebook implements the complete training pipeline for the XR2Text model featuring our novel **HAQT-ARR (Hierarchical Anatomical Query Tokens with Adaptive Region Routing)** projection layer.

## Novel Contribution: HAQT-ARR

Our key innovation is the HAQT-ARR projection layer that bridges vision and language with anatomical awareness:

1. **Hierarchical Anatomical Query Tokens**: Region-specific learnable queries for 7 anatomical regions
2. **Spatial Prior Injection**: Learnable 2D Gaussian priors for anatomical locations
3. **Adaptive Region Routing**: Dynamic weighting of anatomical region importance
4. **Cross-Region Interaction**: Transformer layers modeling inter-region dependencies

## Architecture
```
Input Image (384×384) → Swin Transformer → HAQT-ARR Projection → BioBART Decoder → Report
```

**Authors**: S. Nikhil, Dadhania Omkumar  
**Supervisor**: Dr. Damodar Panigrahy

In [1]:
# ============================================
# GPU/CUDA Check - Run this first!
# ============================================
import os
import sys
sys.path.insert(0, '..')

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['savefig.dpi'] = 300

# GPU Check
print("=" * 50)
print("SYSTEM CONFIGURATION")
print("=" * 50)

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"CUDA Available: True")
    print(f"GPU Connected: {gpu_name}")
    print(f"GPU Memory: {gpu_memory:.1f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"PyTorch Version: {torch.__version__}")
    device = torch.device("cuda")
else:
    print(f"CUDA Available: False")
    print(f"WARNING: Running on CPU (Training will be slow)")
    print(f"PyTorch Version: {torch.__version__}")
    device = torch.device("cpu")

print(f"\nUsing Device: {device}")
print("=" * 50)

F:\MajorProject\swin\lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


SYSTEM CONFIGURATION
CUDA Available: True
GPU Connected: NVIDIA GeForce RTX 4060 Laptop GPU
GPU Memory: 8.0 GB
CUDA Version: 12.1
PyTorch Version: 2.5.1+cu121

Using Device: cuda


## 1. Configuration

In [2]:
# Training Configuration with HAQT-ARR + NOVEL FEATURES
config = {
    # Model
    'image_size': 384,
    'encoder_name': 'base',  # Swin-Base
    'decoder_name': 'biobart',
    'use_anatomical_attention': True,  # Enable HAQT-ARR (Novel)
    
    # HAQT-ARR specific parameters
    'num_regions': 7,              # 7 anatomical regions
    'num_global_queries': 8,       # Global context queries
    'num_region_queries': 4,       # Queries per anatomical region
    'use_spatial_priors': True,    # Learnable Gaussian priors
    'use_adaptive_routing': True,  # Dynamic region weighting
    'use_cross_region': True,      # Cross-region interaction
    
    # Standard parameters
    'language_dim': 768,
    
    # Training
    'epochs': 50,
    'batch_size': 4,
    'gradient_accumulation_steps': 8,  # Effective batch size = 32
    'learning_rate': 5e-5,
    'weight_decay': 0.01,
    'warmup_steps': 500,
    'max_grad_norm': 1.0,
    
    # NOVEL: Novel Loss Functions
    'use_novel_losses': True,
    'use_anatomical_consistency_loss': True,
    'use_clinical_entity_loss': True,
    'use_region_focal_loss': True,
    'use_cross_modal_loss': False,  # Requires decoder hidden states
    'anatomical_loss_weight': 0.1,
    'clinical_loss_weight': 0.2,
    'focal_loss_weight': 0.15,
    'alignment_loss_weight': 0.1,
    
    # NOVEL: Curriculum Learning
    'use_curriculum_learning': True,
    
    # NOVEL: Clinical Validation
    'use_clinical_validation': True,
    
    # Data
    'max_length': 256,
    'num_workers': 2,
    
    # Device
    'use_amp': True,  # Mixed precision for RTX 4060
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    
    # Experiment
    'experiment_name': f'xr2text_haqt_arr_novel_{datetime.now().strftime("%Y%m%d_%H%M%S")}',
    'checkpoint_dir': '../checkpoints',
    'validate_every': 5,  # Validate every 5 epochs (saves time)
    'save_every': 5,  # Save checkpoint every 5 epochs
    'patience': 999,  # Disable early stopping (train all 50 epochs)
    'log_dir': '../logs',
}

# Create directories
os.makedirs(config['checkpoint_dir'], exist_ok=True)
os.makedirs(config['log_dir'], exist_ok=True)
os.makedirs('../data/figures', exist_ok=True)

print("Training Configuration (with HAQT-ARR):")
print("=" * 50)
for k, v in config.items():
    print(f"  {k}: {v}")

Training Configuration (with HAQT-ARR):
  image_size: 384
  encoder_name: base
  decoder_name: biobart
  use_anatomical_attention: True
  num_regions: 7
  num_global_queries: 8
  num_region_queries: 4
  use_spatial_priors: True
  use_adaptive_routing: True
  use_cross_region: True
  language_dim: 768
  epochs: 50
  batch_size: 4
  gradient_accumulation_steps: 8
  learning_rate: 5e-05
  weight_decay: 0.01
  warmup_steps: 500
  max_grad_norm: 1.0
  use_novel_losses: True
  use_anatomical_consistency_loss: True
  use_clinical_entity_loss: True
  use_region_focal_loss: True
  use_cross_modal_loss: False
  anatomical_loss_weight: 0.1
  clinical_loss_weight: 0.2
  focal_loss_weight: 0.15
  alignment_loss_weight: 0.1
  use_curriculum_learning: True
  use_clinical_validation: True
  max_length: 256
  num_workers: 2
  use_amp: True
  device: cuda
  experiment_name: xr2text_haqt_arr_novel_20260107_123159
  checkpoint_dir: ../checkpoints
  validate_every: 5
  save_every: 5
  patience: 999
  log_d

## 2. Load Model and Data

In [3]:
from src.models.xr2text import XR2TextModel, DEFAULT_CONFIG
from src.models.anatomical_attention import ANATOMICAL_REGIONS
from src.data.dataloader import get_dataloaders
from src.utils.device import setup_cuda_optimizations

# Setup CUDA optimizations for RTX 4060
setup_cuda_optimizations()

# Create model with HAQT-ARR (Novel Architecture)
print("Creating XR2Text model with HAQT-ARR projection layer...")
model_config = {
    'image_size': config['image_size'],
    'use_anatomical_attention': config['use_anatomical_attention'],  # Enable HAQT-ARR
    'encoder': {
        'model_name': config['encoder_name'],
        'pretrained': True,
        'freeze_layers': 2,  # Freeze first 2 Swin layers
    },
    'projection': {
        # HAQT-ARR parameters (Novel)
        'language_dim': config['language_dim'],
        'num_regions': config['num_regions'],
        'num_global_queries': config['num_global_queries'],
        'num_region_queries': config['num_region_queries'],
        'use_spatial_priors': config['use_spatial_priors'],
        'use_adaptive_routing': config['use_adaptive_routing'],
        'use_cross_region': config['use_cross_region'],
        'feature_size': 12,  # 384/32 = 12x12 patches
    },
    'decoder': {
        'model_name': config['decoder_name'],
        'max_length': config['max_length'],
    }
}

model = XR2TextModel.from_config(model_config)
model = model.to(config['device'])

# Model summary
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n{'='*50}")
print("XR2Text Model with HAQT-ARR (Novel)")
print(f"{'='*50}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Frozen parameters: {total_params - trainable_params:,}")
print(f"\nAnatomical regions: {model.get_anatomical_regions()}")
print(f"Total queries: {config['num_global_queries'] + config['num_regions'] * config['num_region_queries']}")

2026-01-07 12:32:18.069 | INFO     | src.utils.device:setup_cuda_optimizations:86 - Enabled cuDNN benchmark mode
2026-01-07 12:32:18.072 | INFO     | src.utils.device:setup_cuda_optimizations:92 - Enabled TF32 for matrix operations
2026-01-07 12:32:18.072 | INFO     | src.utils.device:setup_cuda_optimizations:96 - Cleared CUDA cache
2026-01-07 12:32:18.076 | INFO     | src.models.xr2text:__init__:92 - Building Swin Transformer Encoder...
2026-01-07 12:32:18.077 | INFO     | src.models.swin_encoder:__init__:81 - Initializing Swin Encoder: swin_base_patch4_window7_224
2026-01-07 12:32:18.080 | INFO     | src.models.swin_encoder:__init__:82 - Pretrained: True, Image Size: 384


Creating XR2Text model with HAQT-ARR projection layer...


2026-01-07 12:32:21.478 | INFO     | src.models.swin_encoder:__init__:96 - Swin feature dimension: 1024
2026-01-07 12:32:21.483 | INFO     | src.models.swin_encoder:_freeze_layers:136 - Frozen 404,424 parameters in 2 layers
2026-01-07 12:32:21.485 | INFO     | src.models.swin_encoder:__init__:117 - Swin Encoder initialized successfully
2026-01-07 12:32:21.487 | INFO     | src.models.xr2text:__init__:110 - Building HAQT-ARR (Hierarchical Anatomical) Projection Layer...
2026-01-07 12:32:21.489 | INFO     | src.models.anatomical_attention:__init__:601 - Initializing HAQT-ARR Projection Layer
2026-01-07 12:32:21.490 | INFO     | src.models.anatomical_attention:__init__:602 -   Visual dim: 1024 -> Language dim: 768
2026-01-07 12:32:21.492 | INFO     | src.models.anatomical_attention:__init__:603 -   Regions: 7, Total queries: 36
2026-01-07 12:32:21.494 | INFO     | src.models.anatomical_attention:__init__:604 -   Spatial priors: True, Adaptive routing: True
2026-01-07 12:32:21.522 | INFO   


XR2Text Model with HAQT-ARR (Novel)
Total parameters: 251,441,388
Trainable parameters: 251,036,964
Frozen parameters: 404,424

Anatomical regions: ['right_lung', 'left_lung', 'heart', 'mediastinum', 'spine', 'diaphragm', 'costophrenic_angles']
Total queries: 36


In [4]:
# Load data
print("\nLoading datasets...")
tokenizer = model.get_tokenizer()

train_loader, val_loader, test_loader = get_dataloaders(
    tokenizer=tokenizer,
    batch_size=config['batch_size'],
    num_workers=config['num_workers'],
    image_size=config['image_size'],
    max_length=config['max_length'],
    train_subset=None,  # Use full dataset, or set to e.g., 1000 for testing
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

2026-01-07 12:32:25.725 | INFO     | src.data.dataloader:get_dataloaders:47 - Creating dataloaders...
2026-01-07 12:32:25.750 | INFO     | src.data.dataset:__init__:56 - Loading MIMIC-CXR dataset (split: train)...



Loading datasets...


2026-01-07 12:32:29.593 | INFO     | src.data.dataset:__init__:76 - Loaded 30633 samples
2026-01-07 12:32:29.597 | INFO     | src.data.dataset:__init__:56 - Loading MIMIC-CXR dataset (split: validation)...
2026-01-07 12:32:31.272 | INFO     | src.data.dataset:__init__:76 - Loaded 3063 samples
2026-01-07 12:32:31.275 | INFO     | src.data.dataset:__init__:56 - Loading MIMIC-CXR dataset (split: test)...
2026-01-07 12:32:32.888 | INFO     | src.data.dataset:__init__:76 - Loaded 3064 samples
2026-01-07 12:32:32.901 | INFO     | src.data.dataloader:get_dataloaders:117 - Train samples: 30633
2026-01-07 12:32:32.903 | INFO     | src.data.dataloader:get_dataloaders:118 - Val samples: 3063
2026-01-07 12:32:32.906 | INFO     | src.data.dataloader:get_dataloaders:119 - Test samples: 3064
2026-01-07 12:32:32.907 | INFO     | src.data.dataloader:get_dataloaders:120 - Batch size: 4
2026-01-07 12:32:32.909 | INFO     | src.data.dataloader:get_dataloaders:121 - Train batches: 7658


Train batches: 7658
Val batches: 766
Test batches: 766


## 3. Training Setup

In [5]:
from torch.optim import AdamW
from torch.cuda.amp import GradScaler, autocast
from src.training.scheduler import get_cosine_schedule_with_warmup
from src.utils.metrics import compute_metrics

# NOVEL: Import novel training components
from src.training.losses import CombinedNovelLoss
from src.training.curriculum import AnatomicalCurriculumScheduler, create_curriculum_dataloader
from src.utils.clinical_validator import ClinicalValidator

# Optimizer
no_decay = ['bias', 'LayerNorm.weight', 'layer_norm.weight']
optimizer_grouped_parameters = [
    {
        'params': [p for n, p in model.named_parameters() 
                   if p.requires_grad and not any(nd in n for nd in no_decay)],
        'weight_decay': config['weight_decay'],
    },
    {
        'params': [p for n, p in model.named_parameters() 
                   if p.requires_grad and any(nd in n for nd in no_decay)],
        'weight_decay': 0.0,
    },
]

optimizer = AdamW(optimizer_grouped_parameters, lr=config['learning_rate'])

# Scheduler
total_steps = len(train_loader) * config['epochs'] // config['gradient_accumulation_steps']
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=config['warmup_steps'],
    num_training_steps=total_steps,
)

# Mixed precision scaler
scaler = GradScaler() if config['use_amp'] else None

# NOVEL: Initialize novel loss functions
if config.get('use_novel_losses', False):
    novel_loss = CombinedNovelLoss(
        use_anatomical_consistency=config.get('use_anatomical_consistency_loss', True),
        use_clinical_entity=config.get('use_clinical_entity_loss', True),
        use_region_focal=config.get('use_region_focal_loss', True),
        use_cross_modal=config.get('use_cross_modal_loss', False),
        anatomical_weight=config.get('anatomical_loss_weight', 0.1),
        clinical_weight=config.get('clinical_loss_weight', 0.2),
        focal_weight=config.get('focal_loss_weight', 0.15),
        alignment_weight=config.get('alignment_loss_weight', 0.1),
    )
    print("✅ Novel loss functions initialized")
else:
    novel_loss = None

# NOVEL: Initialize curriculum learning scheduler
if config.get('use_curriculum_learning', False):
    curriculum_scheduler = AnatomicalCurriculumScheduler()
    print("✅ Curriculum learning scheduler initialized")
else:
    curriculum_scheduler = None

# NOVEL: Initialize clinical validator
if config.get('use_clinical_validation', False):
    clinical_validator = ClinicalValidator()
    print("✅ Clinical validator initialized")
else:
    clinical_validator = None

print(f"\nTotal optimization steps: {total_steps}")
print(f"Warmup steps: {config['warmup_steps']}")
print(f"Novel losses: {config.get('use_novel_losses', False)}")
print(f"Curriculum learning: {config.get('use_curriculum_learning', False)}")
print(f"Clinical validation: {config.get('use_clinical_validation', False)}")

✅ Novel loss functions initialized
✅ Curriculum learning scheduler initialized
✅ Clinical validator initialized

Total optimization steps: 47862
Warmup steps: 500
Novel losses: True
Curriculum learning: True
Clinical validation: True


## 4. Training Loop

In [6]:
# Training history
history = {
    'train_loss': [],
    'val_loss': [],
    'bleu_1': [],
    'bleu_2': [],
    'bleu_3': [],
    'bleu_4': [],
    'rouge_1': [],
    'rouge_2': [],
    'rouge_l': [],
    'learning_rate': [],
}

best_metric = 0.0
patience_counter = 0
patience = 5

In [ ]:
# Main training loop - Using XR2TextTrainer class
from src.training.trainer import XR2TextTrainer

print("Starting training...")
print("=" * 60)

# Create trainer with all novel features enabled
trainer = XR2TextTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    config=config,
)

# Run training
final_metrics = trainer.train()

# Extract history from trainer for visualization
# The trainer's metrics_tracker stores per-epoch metrics
history = {
    'train_loss': trainer.metrics_tracker.get_history('train_loss'),
    'val_loss': trainer.metrics_tracker.get_history('val_loss'),
    'bleu_1': trainer.metrics_tracker.get_history('bleu_1'),
    'bleu_2': trainer.metrics_tracker.get_history('bleu_2'),
    'bleu_3': trainer.metrics_tracker.get_history('bleu_3'),
    'bleu_4': trainer.metrics_tracker.get_history('bleu_4'),
    'rouge_1': trainer.metrics_tracker.get_history('rouge_1'),
    'rouge_2': trainer.metrics_tracker.get_history('rouge_2'),
    'rouge_l': trainer.metrics_tracker.get_history('rouge_l'),
    'learning_rate': [trainer.scheduler.get_last_lr()[0]] * (trainer.current_epoch + 1),
}

# Add clinical validation metrics if enabled
if config.get('use_clinical_validation', False):
    history['clinical_accuracy'] = trainer.metrics_tracker.get_history('clinical_accuracy')
    history['clinical_f1'] = trainer.metrics_tracker.get_history('clinical_f1')
    history['critical_errors'] = trainer.metrics_tracker.get_history('critical_errors')

# Save training history
history_df = pd.DataFrame(history)
history_df['epoch'] = range(1, len(history_df) + 1)
os.makedirs('../data/statistics', exist_ok=True)
history_df.to_csv('../data/statistics/training_history.csv', index=False)

# Store predictions and references for sample display (will be populated in next cell)
predictions = []
references = []

print("\n" + "=" * 60)
print("TRAINING COMPLETE!")
print("=" * 60)
print(f"\nFinal Metrics:")
for key, value in final_metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

2026-01-07 12:32:33.036 | INFO     | src.utils.device:get_device:27 - Using CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU
2026-01-07 12:32:33.037 | INFO     | src.utils.device:setup_cuda_optimizations:86 - Enabled cuDNN benchmark mode
2026-01-07 12:32:33.039 | INFO     | src.utils.device:setup_cuda_optimizations:92 - Enabled TF32 for matrix operations
2026-01-07 12:32:33.041 | INFO     | src.utils.device:setup_cuda_optimizations:96 - Cleared CUDA cache
2026-01-07 12:32:33.065 | INFO     | src.training.trainer:__init__:130 - Novel loss functions enabled
2026-01-07 12:32:33.067 | INFO     | src.training.trainer:__init__:136 - Curriculum learning enabled
2026-01-07 12:32:33.068 | INFO     | src.training.trainer:__init__:142 - Clinical validation enabled
2026-01-07 12:32:33.080 | INFO     | src.training.trainer:__init__:183 - HAQT-ARR enabled with regions: ['right_lung', 'left_lung', 'heart', 'mediastinum', 'spine', 'diaphragm', 'costophrenic_angles']
2026-01-07 12:32:33.082 | INFO     |

Starting training...


2026-01-07 12:32:33.089 | INFO     | src.training.trainer:__init__:190 - Region regularization: True (weight=0.01)
2026-01-07 12:32:33.089 | INFO     | src.training.trainer:__init__:191 - Early stopping patience: 999
2026-01-07 12:32:33.091 | INFO     | src.training.trainer:train:274 - Starting training...
Epoch 1:   1%|▊                                                                                                                   | 7/958 [00:57<1:02:34,  3.95s/step, loss=15.9440]

## 5. Training Curves Visualization

## 4.5 NOVEL: Enhanced Curriculum Learning Analysis

This section provides detailed analysis of our curriculum learning strategy,
showing how it affects training dynamics and final performance.

In [ ]:
# ============================================
# ENHANCED CURRICULUM LEARNING ANALYSIS
# ============================================
from src.training.curriculum import AnatomicalCurriculumScheduler

print("=" * 80)
print("NOVEL: CURRICULUM LEARNING ANALYSIS")
print("=" * 80)

# Initialize curriculum scheduler
curriculum = AnatomicalCurriculumScheduler()

# Display curriculum stages
print("\n1. CURRICULUM STAGES")
print("-" * 60)
print(f"\n{'Stage':<20} {'Epochs':<15} {'Description':<40}")
print("-" * 80)

stage_descriptions = {
    'normal_cases': 'Normal X-rays, simple findings (e.g., "lungs are clear")',
    'single_region': 'Single anatomical region findings (e.g., cardiomegaly)',
    'multi_region': 'Multiple regions, moderate complexity',
    'complex_cases': 'Complex cases with multiple severe findings',
}

for stage in curriculum.stages:
    name = stage['name']
    epoch_range = f"{stage['epoch_start']}-{stage['epoch_end']}"
    desc = stage_descriptions.get(name, 'Full dataset')
    print(f"{name:<20} {epoch_range:<15} {desc:<40}")

# Curriculum difficulty scoring
print("\n2. SAMPLE DIFFICULTY SCORING")
print("-" * 60)

sample_reports = [
    "Lungs are clear. Heart size is normal. No acute cardiopulmonary process.",
    "Mild cardiomegaly. Lungs are clear bilaterally.",
    "Bilateral pleural effusions. Cardiomegaly. Pulmonary edema.",
    "Large right pneumothorax. Left lung consolidation. Cardiomegaly. Bilateral effusions. ETT in place.",
]

print("\nSample Reports with Difficulty Scores:")
for i, report in enumerate(sample_reports):
    scores = curriculum.difficulty_scorer(report)
    total_difficulty = scores.get('num_findings', 0) + scores.get('severity_score', 0)
    print(f"\n[Sample {i+1}] Difficulty: {total_difficulty:.1f}")
    print(f"   Report: {report[:70]}...")
    print(f"   Findings: {scores.get('num_findings', 0)}, Regions: {scores.get('num_regions', 0)}")

# Simulated curriculum learning results
print("\n3. CURRICULUM LEARNING IMPACT")
print("-" * 60)

# ============================================
# NOTE: Curriculum learning benefits will be measured
# after training completes. The above shows the CONCEPT.
# Real performance comparison will be added post-training.
# ============================================

# NOTE: Curriculum learning benefits will be measured after training.
# Real performance data will be added post-training.

# ============================================
# POST-TRAINING: Curriculum Learning Analysis
# This will show real results after training completes
# ============================================

import os
import pandas as pd
import matplotlib.pyplot as plt

# Check if training history exists
history_path = '../data/statistics/training_history.csv'
if os.path.exists(history_path):
    print("\n" + "=" * 60)
    print("CURRICULUM LEARNING RESULTS (Real Data)")
    print("=" * 60)
    
    # Load training history
    df = pd.read_csv(history_path)
    
    # Analyze curriculum stage transitions
    stage_transitions = [5, 15, 30]  # Epochs where curriculum changes
    
    print("\nPerformance at Curriculum Stage Transitions:")
    print("-" * 60)
    
    for i, epoch in enumerate([1, 5, 15, 30, 50]):
        if epoch <= len(df):
            row = df.iloc[epoch-1]
            stage = ['Stage 1 (Normal)', 'Stage 1→2', 'Stage 2→3', 'Stage 3→4', 'Final'][i]
            print(f"Epoch {epoch} ({stage}):")
            print(f"  BLEU-4: {row['bleu_4']:.4f}")
            print(f"  ROUGE-L: {row['rouge_l']:.4f}")
            print(f"  Loss: {row['val_loss']:.4f}")
            print()
    
    # Plot curriculum impact
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # BLEU-4 progression
    axes[0].plot(df['epoch'], df['bleu_4'], linewidth=2, color='blue')
    for trans in stage_transitions:
        if trans <= len(df):
            axes[0].axvline(x=trans, color='red', linestyle='--', alpha=0.5)
            axes[0].text(trans, axes[0].get_ylim()[1]*0.9, 'Stage\nChange', 
                        ha='center', fontsize=8)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('BLEU-4')
    axes[0].set_title('BLEU-4 Progression with Curriculum Stages')
    axes[0].grid(True, alpha=0.3)
    
    # Loss progression
    axes[1].plot(df['epoch'], df['val_loss'], linewidth=2, color='orange')
    for trans in stage_transitions:
        if trans <= len(df):
            axes[1].axvline(x=trans, color='red', linestyle='--', alpha=0.5)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Validation Loss')
    axes[1].set_title('Loss Progression with Curriculum Stages')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../data/figures/curriculum_impact.png', dpi=300)
    plt.show()
    
    print("✅ Curriculum learning analysis complete!")
    print("   Figure saved: ../data/figures/curriculum_impact.png")
else:
    print("\n⚠️  Training history not found yet.")
    print("   Run this cell again after training completes.")


In [ ]:
# ============================================
# TRAINING CURVES VISUALIZATION
# ============================================
# This works with both freshly trained history AND loaded history from CSV!

# Load history from CSV if not already in memory (i.e., training was skipped)
history_path = "../data/statistics/training_history.csv"
if not history.get('train_loss') or len(history.get('train_loss', [])) == 0:
    if os.path.exists(history_path):
        print("📂 Loading training history from saved CSV...")
        history_df = pd.read_csv(history_path)
        history = history_df.to_dict(orient='list')
        if 'epoch' in history:
            del history['epoch']
        print(f"   Loaded {len(history.get('train_loss', []))} epochs of history")
    else:
        print("⚠️ No training history found!")
else:
    # Save training history if it came from training
    history_df = pd.DataFrame(history)
    history_df['epoch'] = range(1, len(history_df) + 1)
    history_df.to_csv(history_path, index=False)
    print("💾 Training history saved to CSV")

# Check if we have history to plot
if history.get('train_loss') and len(history['train_loss']) > 0:
    # Plot training curves with NOVEL features
    num_plots = 6 if config.get('use_novel_losses', False) or config.get('use_clinical_validation', False) else 4
    fig, axes = plt.subplots(2, 3, figsize=(18, 10)) if num_plots > 4 else plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten() if num_plots > 4 else axes

plot_idx = 0

# Loss curves
axes[plot_idx].plot(history['train_loss'], label='Train Loss', color='blue', linewidth=2)
axes[plot_idx].plot(history['val_loss'], label='Val Loss', color='orange', linewidth=2)
axes[plot_idx].set_xlabel('Epoch')
axes[plot_idx].set_ylabel('Loss')
axes[plot_idx].set_title('Training and Validation Loss')
axes[plot_idx].legend()
axes[plot_idx].grid(True, alpha=0.3)
plot_idx += 1

# BLEU scores
axes[plot_idx].plot(history['bleu_1'], label='BLEU-1', linewidth=2)
axes[plot_idx].plot(history['bleu_2'], label='BLEU-2', linewidth=2)
axes[plot_idx].plot(history['bleu_3'], label='BLEU-3', linewidth=2)
axes[plot_idx].plot(history['bleu_4'], label='BLEU-4', linewidth=2)
axes[plot_idx].set_xlabel('Epoch')
axes[plot_idx].set_ylabel('Score')
axes[plot_idx].set_title('BLEU Scores')
axes[plot_idx].legend()
axes[plot_idx].grid(True, alpha=0.3)
plot_idx += 1

# ROUGE scores
axes[plot_idx].plot(history['rouge_1'], label='ROUGE-1', linewidth=2)
axes[plot_idx].plot(history['rouge_2'], label='ROUGE-2', linewidth=2)
axes[plot_idx].plot(history['rouge_l'], label='ROUGE-L', linewidth=2)
axes[plot_idx].set_xlabel('Epoch')
axes[plot_idx].set_ylabel('Score')
axes[plot_idx].set_title('ROUGE Scores')
axes[plot_idx].legend()
axes[plot_idx].grid(True, alpha=0.3)
plot_idx += 1

# NOVEL: Novel loss components
if config.get('use_novel_losses', False) and 'anatomical_consistency_loss' in history:
    axes[plot_idx].plot(history['anatomical_consistency_loss'], label='Anatomical Consistency', linewidth=2, color='purple')
    axes[plot_idx].plot(history['clinical_entity_loss'], label='Clinical Entity', linewidth=2, color='red')
    axes[plot_idx].plot(history['region_focal_loss'], label='Region Focal', linewidth=2, color='green')
    axes[plot_idx].set_xlabel('Epoch')
    axes[plot_idx].set_ylabel('Loss')
    axes[plot_idx].set_title('Novel Loss Components (NOVEL)')
    axes[plot_idx].legend()
    axes[plot_idx].grid(True, alpha=0.3)
    plot_idx += 1

# NOVEL: Clinical validation metrics
if config.get('use_clinical_validation', False) and 'clinical_accuracy' in history:
    ax_twin = axes[plot_idx].twinx()
    axes[plot_idx].plot(history['clinical_accuracy'], label='Clinical Accuracy', linewidth=2, color='blue')
    axes[plot_idx].plot(history['clinical_f1'], label='Clinical F1', linewidth=2, color='orange')
    axes[plot_idx].set_xlabel('Epoch')
    axes[plot_idx].set_ylabel('Score', color='black')
    axes[plot_idx].set_title('Clinical Validation Metrics (NOVEL)')
    axes[plot_idx].legend(loc='upper left')
    axes[plot_idx].grid(True, alpha=0.3)
    
    # Critical errors on secondary axis
    ax_twin.plot(history['critical_errors'], label='Critical Errors', linewidth=2, color='red', linestyle='--')
    ax_twin.set_ylabel('Critical Errors', color='red')
    ax_twin.legend(loc='upper right')
    ax_twin.tick_params(axis='y', labelcolor='red')
    plot_idx += 1

# Learning rate
axes[plot_idx].plot(history['learning_rate'], color='green', linewidth=2)
axes[plot_idx].set_xlabel('Epoch')
axes[plot_idx].set_ylabel('Learning Rate')
axes[plot_idx].set_title('Learning Rate Schedule')
axes[plot_idx].set_yscale('log')
axes[plot_idx].grid(True, alpha=0.3)

# Hide unused subplots
for i in range(plot_idx + 1, len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.savefig('../data/figures/training_curves_novel.png', dpi=300)
plt.show()
print("✅ Training curves saved with NOVEL features visualization")

## 6. Sample Predictions

In [ ]:
# Show sample predictions vs ground truth
print("Sample Predictions vs Ground Truth:")
print("=" * 80)

for i in range(min(5, len(predictions))):
    print(f"\n--- Sample {i+1} ---")
    print(f"\nGround Truth:")
    print(references[i][:500] + "..." if len(references[i]) > 500 else references[i])
    print(f"\nGenerated:")
    print(predictions[i][:500] + "..." if len(predictions[i]) > 500 else predictions[i])
    print("-" * 80)

## 7. Final Results Summary

In [ ]:
# Best results
best_epoch = np.argmax([h['bleu_4'] + h['rouge_l'] for h in [dict(zip(history.keys(), v)) for v in zip(*history.values())]])

print("=" * 60)
print("TRAINING RESULTS SUMMARY")
print("=" * 60)
print(f"\nBest Epoch: {best_epoch + 1}")
print(f"\nBest Metrics:")
print(f"  BLEU-1: {history['bleu_1'][best_epoch]:.4f}")
print(f"  BLEU-2: {history['bleu_2'][best_epoch]:.4f}")
print(f"  BLEU-3: {history['bleu_3'][best_epoch]:.4f}")
print(f"  BLEU-4: {history['bleu_4'][best_epoch]:.4f}")
print(f"  ROUGE-1: {history['rouge_1'][best_epoch]:.4f}")
print(f"  ROUGE-2: {history['rouge_2'][best_epoch]:.4f}")
print(f"  ROUGE-L: {history['rouge_l'][best_epoch]:.4f}")
print(f"\nFinal Train Loss: {history['train_loss'][-1]:.4f}")
print(f"Final Val Loss: {history['val_loss'][-1]:.4f}")

# Save results table
results_table = pd.DataFrame({
    'Metric': ['BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L'],
    'Score': [
        history['bleu_1'][best_epoch],
        history['bleu_2'][best_epoch],
        history['bleu_3'][best_epoch],
        history['bleu_4'][best_epoch],
        history['rouge_1'][best_epoch],
        history['rouge_2'][best_epoch],
        history['rouge_l'][best_epoch],
    ]
})
results_table.to_csv('../data/statistics/best_results.csv', index=False)
print("\nResults saved to ../data/statistics/best_results.csv")